In [2]:
pip install ipyleaflet

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=ab52e44761f8b88b2d97189290d11c5bfa80b3d4a3fea39414286b497c71ad47
  Stored in directory: c:\users\morri\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import packages
import IPython
import ipywidgets
import ipyleaflet
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import random
import os
import tensorflow as tf

import descarteslabs as dl
import descarteslabs.workflows as wf

from utils import make_ground_dataset_from_polygons, get_center_location
from unet import UNet

ModuleNotFoundError: No module named 'tensorflow'

In [30]:
#import json
with open("plane.geojson") as f:
    fc = json.load(f)

In [31]:
#how many polygons?
len(fc["features"])

5

In [42]:
n_samples = make_ground_dataset_from_polygons(
    "plane.geojson",
    products="airbus:oneatlas:phr:v2",
    bands=["red", "green", "blue"],
    resolution=10,
    tilesize=64,
    pad=0,
    start_datetime="2020-03-19",
    end_datetime="2020-04-24",
    out_file="plane.tfrecords",
)

Processing 5 polygons.
Create TFRecords for 2 unique dltiles that cover these polygons.
Caught exception for dltile 64:0:10.0:12:23:5557 - from_key() got an unexpected keyword argument 'raster_client'
Caught exception for dltile 64:0:10.0:12:23:5557 - from_key() got an unexpected keyword argument 'raster_client'
Caught exception for dltile 64:0:10.0:12:23:5557 - from_key() got an unexpected keyword argument 'raster_client'
Caught exception for dltile 64:0:10.0:12:23:5558 - from_key() got an unexpected keyword argument 'raster_client'
Caught exception for dltile 64:0:10.0:12:23:5558 - from_key() got an unexpected keyword argument 'raster_client'
Caught exception for dltile 64:0:10.0:12:23:5558 - from_key() got an unexpected keyword argument 'raster_client'


In [43]:
# Define the features in the TFRecords file
features = {
    "image/image_data": tf.io.FixedLenSequenceFeature(
        [], dtype=tf.float32, allow_missing=True
    ),
    "image/height": tf.io.FixedLenFeature([], tf.int64),
    "image/width": tf.io.FixedLenFeature([], tf.int64),
    "image/channels": tf.io.FixedLenFeature([], tf.int64),
    "target/target_data": tf.io.FixedLenSequenceFeature(
        [], dtype=tf.float32, allow_missing=True
    ),
    "target/height": tf.io.FixedLenFeature([], tf.int64),
    "target/width": tf.io.FixedLenFeature([], tf.int64),
    "target/channels": tf.io.FixedLenFeature([], tf.int64),
    "dltile": tf.io.FixedLenFeature([], tf.string),
}

In [44]:
def parse_example(example_proto):
    image_features = tf.io.parse_single_example(example_proto, features)

    img_height = tf.cast(image_features["image/height"], tf.int32)
    img_width = tf.cast(image_features["image/width"], tf.int32)
    img_channels = tf.cast(image_features["image/channels"], tf.int32)

    target_height = tf.cast(image_features["target/height"], tf.int32)
    target_width = tf.cast(image_features["target/width"], tf.int32)
    target_channels = tf.cast(image_features["target/channels"], tf.int32)

    image_raw = tf.reshape(
        tf.squeeze(image_features["image/image_data"]),
        tf.stack([img_height, img_width, img_channels]),
    )

    target_raw = tf.reshape(
        tf.squeeze(image_features["target/target_data"]),
        tf.stack([target_height, target_width, target_channels]),
    )

    return image_raw, target_raw

In [45]:
# Create a TFRecordDataset to read images from these TFRecords
data = tf.data.TFRecordDataset("plane.tfrecords").map(
    parse_example, num_parallel_calls=4
)

data_viz = iter(data.batch(1))

In [46]:
# Visualize samples. You can re-run this cell to iterate through the dataset.
img, trg = next(data_viz)
fig, ax = plt.subplots(1, 2, figsize=(8, 5))
ax[0].imshow(img.numpy().astype(np.uint8)[0])
ax[1].imshow(trg.numpy().astype(np.uint8)[0].squeeze())

StopIteration: 

In [47]:
# Build the model. We could just use the base_model but then the input size would be fixed
# once we load a saved model. In order to be able to predict on larger tiles we create an
# input layer with no fixed size
base_model = UNet()
inputs = tf.keras.layers.Input(shape=(None, None, 3))
model = tf.keras.Model(inputs=inputs, outputs=base_model(inputs))

In [48]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [49]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
u_net_2 (UNet)               (None, None, None, 1)     7760097   
Total params: 7,760,097
Trainable params: 7,760,097
Non-trainable params: 0
_________________________________________________________________


In [60]:
transforms = [type_transform, subtract_mean]
batch_size = 8
n_samples = 271
n_train_samples = int(0.8 * n_samples)
n_val_samples = n_samples - n_train_samples

data = tf.data.TFRecordDataset("plane.tfrecords").map(
    parse_example, num_parallel_calls=4
)

# Apply transforms
for t in transforms:
    data = data.map(t, num_parallel_calls=4)

# Shuffle the data and split into train and validation set
data = data.shuffle(buffer_size=1000)
data_train = data.take(n_train_samples).repeat().batch(batch_size)
data_val = data.skip(n_train_samples).repeat().batch(batch_size)

In [62]:
history = model.fit(
    data_train,
    steps_per_epoch=n_train_samples // batch_size,
    validation_data=data_val,
    validation_steps=n_val_samples // batch_size,
    epochs=100,
)

Epoch 1/100


UnboundLocalError: local variable 'logs' referenced before assignment